# Create a time-of-flight lookup table for DREAM

In [ ]:
import scipp as sc
import sciline as sl
from ess.reduce import time_of_flight
from ess.dream.beamline import InstrumentConfiguration, choppers

## Select the choppers

In [ ]:
disk_choppers = choppers(InstrumentConfiguration.high_flux)

## Setting up the workflow

In [ ]:
wf = sl.Pipeline(
    time_of_flight.providers(), params=time_of_flight.default_parameters()
)

wf[time_of_flight.LtotalRange] = sc.scalar(60.0, unit="m"), sc.scalar(80.0, unit="m")
wf[time_of_flight.SimulationResults] = time_of_flight.simulate_beamline(
    choppers=disk_choppers, neutrons=5_000_000, source_position=sc.vector([0, 0, 0], unit='m'),
)

wf[time_of_flight.DistanceResolution] = sc.scalar(0.1, unit="m")
wf[time_of_flight.TimeResolution] = sc.scalar(250.0, unit='us')
wf[time_of_flight.LookupTableRelativeErrorThreshold] = 0.02
wf[time_of_flight.PulsePeriod] = 1.0 / sc.scalar(14.0, unit="Hz")
wf[time_of_flight.PulseStride] = 1
wf[time_of_flight.PulseStrideOffset] = None

## Compute the table

In [ ]:
table = wf.compute(time_of_flight.TimeOfFlightLookupTable)
table

In [ ]:
table.plot()

## Save to file

In [ ]:
# Save chopper metadata
# TODO: storing the choppers as a PyObject is skipped when saving to disk
table.coords['choppers'] = sc.scalar(disk_choppers)
# Write to file
table.save_hdf5('DREAM-high-flux-tof-lookup-table.h5')